In [1]:
import gym
import pylab
import random
import numpy as np
from collections import deque
import tflearn

## Task: fill empty spaces in the following agent code

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam

Using TensorFlow backend.


In [3]:
class DeepQAgent:
    def __init__(self, state_size, action_size, render=False):
        # Tip: if you are training this on AWS the best way is to turn off rendering
        # and load it later with the serialized model
        self.render = render
        self.state_size = state_size
        self.action_size = action_size

        self.discount_factor = 0.99
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_min = 0.005
        self.epsilon_decay = (self.epsilon - self.epsilon_min) / 50000
        self.batch_size = 64
        self.train_start = 1000
        # replay memory
        self.memory = deque(maxlen=10000)

        self.model = self.build_model()
        self.target_model = self.build_model()
        self.update_target_model()

    def build_model(self):
        # Use tflearn to get simple NN for deep q-learning
        # Spoler alert: a couple of fully connected hidden layers should be enough
        # Output layer should have the same dimensionality as the action space
        # TODO
        
        model = Sequential()
        model.add(Dense(32, activation='relu', input_dim = self.state_size))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(self.action_size))
        model.compile(optimizer = Adam(lr=self.learning_rate), loss="mse")
        return model

    def update_target_model(self):
        """Update your target model to the model you are currently learning at regular time intervals"""
        self.target_model.set_weights(self.model.get_weights())

    def get_action(self, state):
        """The choice of action uses the epsilon-greedy policy for the current network."""
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            q_value = self.model.predict(state)
            return np.argmax(q_value[0])

    def replay_memory(self, state, action, reward, next_state, done):
        """Save <s, a, r, s'> to replay_memory"""
        if action == 2:
            action = 1
        self.memory.append((state, action, reward, next_state, done))
        if self.epsilon > self.epsilon_min:
            self.epsilon -= self.epsilon_decay
            # print(len(self.memory))

    def train_replay(self):
        """Random sampling of batch_size samples from replay memory"""
        if len(self.memory) < self.train_start:
            return
        batch_size = min(self.batch_size, len(self.memory))
        mini_batch = random.sample(self.memory, batch_size)

        update_input = np.zeros((batch_size, self.state_size))
        update_target = np.zeros((batch_size, self.action_size))

        for i in range(batch_size):
            state, action, reward, next_state, done = mini_batch[i]
            target = self.model.predict(state)[0]

            # As in queuing, it gets the maximum Q Value at s'. However, it is imported from the target model.
            if done:
                target[action] = reward
            else:
                target[action] = reward + self.discount_factor * \
                                          np.amax(self.target_model.predict(next_state)[0])
            update_input[i] = state
            update_target[i] = target

        # You can create a minibatch of the correct target answer and the current value of your own,
        self.model.fit(update_input, update_target, batch_size=batch_size, epochs=1, verbose=0)

    def load_model(self, name):
        # TODO
        self.model.load_model(name)

    def save_model(self, name):
        # TODO
        self.model.save(name)


In [4]:
env = gym.make('MountainCar-v0')
state_size = env.observation_space.shape[0] # should be equal 2
ACTION_SIZE = 2
agent = DeepQAgent(state_size, ACTION_SIZE)
# agent.load_model("./save_model/<your_saved_model_name>")
scores, episodes = [], []
N_EPISODES = 4000

In [5]:
for e in range(N_EPISODES):
    done = False
    score = 0
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    print(state)

    # Action 0 (left), 1 (do nothing), 3 (declare fake_action to avoid doing nothing
    fake_action = 0

    # Counter for the same action 4 times
    action_count = 0

    while not done:
        if agent.render:
            env.render()

        # Select an action in the current state and proceed to a step
        action_count = action_count + 1

        if action_count == 4:
            action = agent.get_action(state)
            action_count = 0

            if action == 0:
                fake_action = 0
            elif action == 1:
                fake_action = 2

        # Take 1 step with the selected action
        next_state, reward, done, info = env.step(fake_action)
        next_state = np.reshape(next_state, [1, state_size])
        # Give a penalty of -100 for actions that end an episode
        # reward = reward if not done else -100

        # Save <s, a, r, s'> to replay memory
        agent.replay_memory(state, fake_action, reward, next_state, done)
        # Continue to learn every time step
        agent.train_replay()
        score += reward
        state = next_state

        if done:
            env.reset()
            # Copy the learning model for each episode to the target model
            agent.update_target_model()

            # For each episode, the time step where cartpole stood is plot
            scores.append(score)
            episodes.append(e)
            print("episode:", e, "  score:", score, "  memory length:", len(agent.memory),
                  "  epsilon:", agent.epsilon)

    # Save model for every 50 episodes
    if e % 50 == 0:
        agent.save_model("./save_model/model_32")

[[-0.48671628  0.        ]]
episode: 0   score: -200.0   memory length: 200   epsilon: 0.9960200000000077
[[-0.58571944  0.        ]]
episode: 1   score: -200.0   memory length: 400   epsilon: 0.9920400000000154
[[-0.45345973  0.        ]]
episode: 2   score: -200.0   memory length: 600   epsilon: 0.988060000000023
[[-0.40139476  0.        ]]
episode: 3   score: -200.0   memory length: 800   epsilon: 0.9840800000000307
[[-0.5104905  0.       ]]
episode: 4   score: -200.0   memory length: 1000   epsilon: 0.9801000000000384
[[-0.57890589  0.        ]]
episode: 5   score: -200.0   memory length: 1200   epsilon: 0.9761200000000461
[[-0.55384785  0.        ]]
episode: 6   score: -200.0   memory length: 1400   epsilon: 0.9721400000000537
[[-0.47016958  0.        ]]
episode: 7   score: -200.0   memory length: 1600   epsilon: 0.9681600000000614
[[-0.49560149  0.        ]]
episode: 8   score: -200.0   memory length: 1800   epsilon: 0.9641800000000691
[[-0.42931667  0.        ]]
episode: 9   sco

episode: 76   score: -200.0   memory length: 10000   epsilon: 0.6936793000005909
[[-0.53853868  0.        ]]
episode: 77   score: -200.0   memory length: 10000   epsilon: 0.6896993000005985
[[-0.46640477  0.        ]]
episode: 78   score: -200.0   memory length: 10000   epsilon: 0.6857193000006062
[[-0.56747364  0.        ]]
episode: 79   score: -200.0   memory length: 10000   epsilon: 0.6817393000006139
[[-0.55809304  0.        ]]
episode: 80   score: -200.0   memory length: 10000   epsilon: 0.6777593000006216
[[-0.58020558  0.        ]]
episode: 81   score: -200.0   memory length: 10000   epsilon: 0.6737793000006292
[[-0.57401484  0.        ]]
episode: 82   score: -200.0   memory length: 10000   epsilon: 0.6697993000006369
[[-0.49760603  0.        ]]
episode: 83   score: -200.0   memory length: 10000   epsilon: 0.6658193000006446
[[-0.46539941  0.        ]]
episode: 84   score: -200.0   memory length: 10000   epsilon: 0.6618393000006523
[[-0.55367165  0.        ]]
episode: 85   score

episode: 151   score: -200.0   memory length: 10000   epsilon: 0.40369650000088153
[[-0.40686147  0.        ]]
episode: 152   score: -200.0   memory length: 10000   epsilon: 0.3997165000008781
[[-0.53539603  0.        ]]
episode: 153   score: -200.0   memory length: 10000   epsilon: 0.3957365000008747
[[-0.57253645  0.        ]]
episode: 154   score: -200.0   memory length: 10000   epsilon: 0.39175650000087126
[[-0.51188932  0.        ]]
episode: 155   score: -200.0   memory length: 10000   epsilon: 0.38777650000086783
[[-0.41602122  0.        ]]
episode: 156   score: -200.0   memory length: 10000   epsilon: 0.3837965000008644
[[-0.53358417  0.        ]]
episode: 157   score: -200.0   memory length: 10000   epsilon: 0.379816500000861
[[-0.4924847  0.       ]]
episode: 158   score: -200.0   memory length: 10000   epsilon: 0.37583650000085755
[[-0.50352566  0.        ]]
episode: 159   score: -200.0   memory length: 10000   epsilon: 0.37185650000085413
[[-0.56974749  0.        ]]
episode:

episode: 226   score: -200.0   memory length: 10000   epsilon: 0.14933470000080304
[[-0.48897888  0.        ]]
episode: 227   score: -155.0   memory length: 10000   epsilon: 0.1462502000008047
[[-0.55162787  0.        ]]
episode: 228   score: -169.0   memory length: 10000   epsilon: 0.14288710000080648
[[-0.56069233  0.        ]]
episode: 229   score: -122.0   memory length: 10000   epsilon: 0.14045930000080778
[[-0.56242878  0.        ]]
episode: 230   score: -115.0   memory length: 10000   epsilon: 0.138170800000809
[[-0.5010188  0.       ]]
episode: 231   score: -159.0   memory length: 10000   epsilon: 0.1350067000008107
[[-0.47686057  0.        ]]
episode: 232   score: -137.0   memory length: 10000   epsilon: 0.13228040000081215
[[-0.50783547  0.        ]]
episode: 233   score: -149.0   memory length: 10000   epsilon: 0.12931530000081373
[[-0.56570942  0.        ]]
episode: 234   score: -200.0   memory length: 10000   epsilon: 0.12533530000081586
[[-0.56805909  0.        ]]
episode

KeyboardInterrupt: 